In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [22]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/',
    search_pattern_re='cam4.*avi',
    max_depth=5,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230531/cam42023-05-31T12_59_07.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230523/cam42023-05-23T12_42_31.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230517/cam42023-05-17T13_16_31.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230515/cam42023-05-15T11_32_19.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230430/cam42023-04-30T13_04_58.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230528/cam42023-05-28T09_46_31.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230601/cam42023-06-01T13_31_14.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230512/cam4

In [23]:
dates_toUse = natsort.natsorted([Path(p).parts[-2] for p in paths_found])
dates_toUse

['20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510',
 '20230511',
 '20230512',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230528',
 '20230529',
 '20230530',
 '20230531',
 '20230601']

In [24]:
paths_vids = {d: v for d, v in zip(dates_toUse, paths_found)}

## Start a small interactive node

In [25]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

slurmstepd: error: *** STEP 17122172.0 ON compute-e-16-236 CANCELLED AT 2023-09-07T21:30:23 DUE TO TIME LIMIT ***
srun: Job step aborted: Waiting up to 62 seconds for job step to finish.
srun: error: compute-e-16-236: task 0: Killed
(base) [rh183@login04 ~]$ 


## Pull/update repo

In [26]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

bash --pty -p interactive -t 0-00:30:00 --mem=4G /bin/ 
srun: job 17145016 queued and waiting for resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP

srun: job 17145016 has been allocated resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP

(base) [rh183@compute-a-16-169 ~]$ cd /n/data1/
_repos/NBAPo/sabatini/rich/github 
(base) [rh183@compute-a-16-169 NBAP]$ 


## Activate environment

In [27]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

git pull

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Already up-to-date.

ini/rich/virtual_envs/FR16-169 NBAP]$ conda activate /n/data1/hms/neurobio/sabat 



## Prepare commands

In [28]:
# duration_sessions = {date: 60*60 if float(date) < float(20230725) else 80*60 for date in dates_toUse}

duration_sessions = {date: 60*60 for date in dates_toUse}

In [29]:
for path_vid in paths_found[:]:

    mouse = Path(path_vid).parts[-4]
    date = Path(path_vid).parts[-2]
    
    name_slurm = mouse[-3:] + '_' + date[-4:]

    path_vid = path_vid
    path_mask = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / date / ('mask_' + Path(path_vid).stem + '.npy') )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / date )

    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env

    time_fastForward = duration_sessions[date] - 1
    
    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {path_vid} \
    {path_mask} \
    {name_job} \
    {name_slurm} \
    {name_env}\
    {time_fastForward}\
    ",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230430/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230430     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230430/cam42023-04-30T13_04_58.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230430/mask_cam42023-04-30T13_04_58.npy     jobNum_     22N_0430     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-169 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230501/cam42023-05-01T10_48_48.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501/mask_cam42023-05-01T10_48_48.npy     jobNum_     22N_0501     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

       22N_0430     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599  

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230501/cam42023-05-01T10_48_48.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230501/mask_cam42023-05-01T10_48_48.npy     jobNum_     22N_0501     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extr

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230502/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230502     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230502/cam42023-05-02T12_33_29.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230502/mask_cam42023-05-02T12_33_29.npy     jobNum_     22N_0502     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

...

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230530/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230530     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230530/cam42023-05-30T13_06_25.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230530/mask_cam42023-05-30T13_06_25.npy     jobNum_     22N_0530     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py to /n/data1/hms/neurobio/sabatini/rich/analysis/eye_las

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230531/cam42023-05-31T12_59_07.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531/mask_cam42023-05-31T12_59_07.npy     jobNum_     22N_0531     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230531/cam42023-05-31T12_59_07.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230531/mask_cam42023-05-31T12_59_07.npy     jobNum_     22N_0531     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py to /n/data1/hms/neurobio/sabatini/rich/analysis/eye_las

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230601/cam42023-06-01T13_31_14.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601/mask_cam42023-06-01T13_31_14.npy     jobNum_     22N_0601     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

       22N_0531     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599  
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230601/cam42023-06-01T13_31_14.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322N/20230601/mask_cam42023-06-01T13_31_14.npy     jobNum_     22N_0601     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extra

### check on job

### Check if it worked

In [85]:
test = natsort.natsorted(sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R',
    search_pattern_re=r'idx_eye_laser.pkl',
))
display(test)

/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230531/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230420/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230523/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230517/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230515/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230430/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230512/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230511/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/anal

['/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230419/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230420/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230424/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230425/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230426/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230427/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230428/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0322R/20230429/jobNum_0/idx_eye_laser.pkl',
 '/n/dat

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()